In [ ]:
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
import time


# basic setup
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300)
detections = model(camera.value)
print(detections)
robot = Robot()

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

target_obj_tall = 21
target_obj_short = 31

obstacle_obj_1 = 3
obstacel_obj_2 = 8

object_info_dict = {21:("tall_object", 200), 31:("short_object", 40), 3:("Apple", 30), 8:("Banana", 40)}

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def estimate_distance(detection):
    focal_length = 105.0  #need to caliborate your camera to get this
    obj_info = object_info_dict[detection['label']]
    bbox = detection['bbox']  
    obj_width = obj_info[1]
    obj_width_in_img = width * (bbox[2] - bbox[0])        
    dist = obj_width * focal_length / obj_width_in_img
    return dist


is_local_steering_in_progress = False

def local_steering(detections):
    # check if local steering is needed based on distance
    is_local_steering_needed = false

    if(is_local_steering_needed)
        is_local_steering_in_progress = True
        #local_steering code come here
    
    
        is_local_steering_in_progress = False
    
    
    
def is_target_reached(detections):
    # check if the car is close enough to the target
    return False
    
    
def lock_and_move_to_target(detections):
    # detect the target
    
    # if not visible, may need to rotate the car 
    
    # move toward the object


In [ ]:
def execute(change):
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    # visualized target and obstacle objects
    for det in detections[0]:
        bbox = det['bbox']
        label = det['label']
        dist = estimate_distance(detection)
        if(label == target_obj_tall || label == target_obj_short):
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, obj_info_dict[label][0] + ":" + str(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
            
        if(label == obstacle_obj_1 || label == obstacle_obj_2)
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
            cv2.putText(image, obj_info_dict[label][0] + ":" + str(dist), (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),2)
        
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
    # check if target is reached or not
    if(is_target_reached(detections)):
        camera.unobserve_all()
        time.sleep(1.0)
        robot.stop()
        return
    
    # if in local steering mode, return
    if(is_local_steering_in_progress):
        return
    
    # call local_steering to decide if local_steering is needed
    # if needed, perform local steering
    local_steering(detections)
        
   
    # if not in local steering mode, move to target
    if(is_local_steering_in_progress == False):
        lock_and_move_to_target(detections)
    

    
execute({'new': camera.value})

camera.unobserve_all()
camera.observe(execute, names='value')

In [ ]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()